# 4.1 Analysis-stats

- premiers tests à l'arrache

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv(
    "../data/interim/df_repu.csv", low_memory=False, dtype={"ID_orateur": str}
)

In [3]:
df.shape

(3668, 61)

### dynamique temporelle

In [4]:
df["DateSeance_ts"] = pd.to_datetime(df["DateSeance"], format="%Y%m%d%H%M%S%f")
df["DateSeance_day"] = df["DateSeance_ts"].dt.normalize()  # guess it works


In [5]:
fig_time = px.bar(df.resample("W", on="DateSeance_day").size())

fig_time.update_layout(
    title="Nombre de mentions de la notion de république",  # ajouter un titre
    xaxis_title="Date",
    yaxis_title="Nombre de mentions",  # renommer les étiquettes d'axes
    template="plotly_white",  # changer le style du graphique
    showlegend=False,
)  # masquer la légende

# Afficher le graphique
fig_time.show()

/Users/leo/anaconda3/envs/myenv_clone/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [6]:
fig_time.write_html("../reports/figs/fig_time.html")

In [7]:
# aficher les 25 dates les plus fréquentes sous forme de tableau
table = df["DateSeance_day"].value_counts()[0:20].reset_index()
table = table.rename(columns={"count": "Nombre de mentions"})
table

,DateSeance_day,Nombre de mentions
0,2023-12-07,93
1,2023-04-05,90
2,2023-01-12,71
3,2023-07-04,68
4,2022-11-18,66
5,2023-07-11,59
6,2023-05-10,59
7,2024-03-06,54
8,2023-12-19,51
9,2024-01-17,49


### groupes

In [8]:
df["groupeAbrev"].value_counts()

groupeAbrev
EPR          641
LFI-NFP      523
ECOS         414
RN           289
RE           237
SOC          224
DEM          173
DR           162
HOR          138
GDR          129
LIOT         100
GDR-NUPES     92
LFI-NUPES     86
NI            78
LR            45
SOC-A         21
UDR           20
LAREM         11
ECOLO          3
Name: count, dtype: int64

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Calculer les mentions par groupe et par jour
df_grouped = (
    df.groupby([pd.Grouper(key="DateSeance_day", freq="W"), "groupeAbrev"])
    .size()
    .reset_index(name="mentions")
)

# Trier les groupes par nombre total de mentions
counts = df["groupeAbrev"].value_counts()
groupes = counts.index.tolist()

cols = 4
rows = (len(groupes) + cols - 1) // cols  # nombre de lignes nécessaires

# Calcul de la valeur maximale pour fixer la même échelle Y
max_y = df_grouped["mentions"].max()

fig_group = make_subplots(rows=rows, cols=cols, shared_xaxes=True, subplot_titles=groupes)

for idx, groupe in enumerate(groupes):
    row = idx // cols + 1
    col = idx % cols + 1

    data_groupe = df_grouped[df_grouped["groupeAbrev"] == groupe]
    fig_group.add_trace(
        go.Bar(x=data_groupe["DateSeance_day"], y=data_groupe["mentions"], name=groupe),
        row=row,
        col=col,
    )

fig_group.update_layout(
    height=300 * rows,
    width=1200,
    title_text="Dynamique temporelle des mentions de l'idée de république par groupe parlementaire",
    showlegend=False,
    template="plotly_white",
)

for row in range(1, rows + 1):
    for col in range(1, cols + 1):
        fig_group.update_yaxes(range=[0, max_y], row=row, col=col)

for col in range(1, cols + 1):
    fig_group.update_xaxes(title_text="Date", row=rows, col=col)

for row in range(1, rows + 1):
    fig_group.update_yaxes(title_text="Nombre de mentions", row=row, col=1)

fig_group.show()


/Users/leo/anaconda3/envs/myenv_clone/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [10]:
fig_group.write_html("../reports/figs/fig_group.html")

### parlementaires

In [11]:
df["Nom_orateur"].value_counts()[0:20]

Nom_orateur
M. Benjamin Lucas           166
M. Gérald Darmanin          160
Mme Élisabeth Borne         130
M. Antoine Léaument          92
M. Gabriel Attal             63
M. Éric Dupond-Moretti       62
Mme Mathilde Panot           52
Mme Aurore Bergé             38
Mme Sonia Backès             36
M. Sébastien Lecornu         36
M. Boris Vallaud             35
M. Jean-Philippe Tanguy      31
M. Nicolas Sansu             31
M. Arthur Delaporte          31
M. Sébastien Jumel           30
M. Alexis Corbière           30
M. Pierre Dharréville        29
Mme Cyrielle Chatelain       29
M. Mathieu Lefèvre           28
M. Jean-François Carenco     28
Name: count, dtype: int64

In [12]:
import plotly.express as px

fig_top_orateurs = px.bar(
    df["Nom_orateur"].value_counts()[0:10],
    # x=top_counts.index,
    # y=top_counts.values,
    labels={"value": "Nombre de mentions", "Nom_orateur": "Orateur"},
    title="Top 20 orateurs par nombre de mentions",
    template="plotly_white",
)
fig_top_orateurs.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
)
fig_top_orateurs.show()

In [13]:
fig_top_orateurs.write_html("../reports/figs/fig_top_orateurs.html")

In [14]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Top 20 orateurs
top_orateurs = df["Nom_orateur"].value_counts().index[:20].tolist()

# Grouper par semaine et orateur
df_orateur = df[df["Nom_orateur"].isin(top_orateurs)]
df_grouped_orateur = (
    df_orateur.groupby([pd.Grouper(key="DateSeance_day", freq="MS"), "Nom_orateur"])
    .size()
    .reset_index(name="mentions")
)

cols = 4
rows = (len(top_orateurs) + cols - 1) // cols
max_y = df_grouped_orateur["mentions"].max()

fig_orateurs_time = make_subplots(
    rows=rows, cols=cols, shared_xaxes=True, subplot_titles=top_orateurs
)

for idx, orateur in enumerate(top_orateurs):
    row = idx // cols + 1
    col = idx % cols + 1
    data_orateur = df_grouped_orateur[df_grouped_orateur["Nom_orateur"] == orateur]
    fig_orateurs_time.add_trace(
        go.Bar(
            x=data_orateur["DateSeance_day"], y=data_orateur["mentions"], name=orateur
        ),
        row=row,
        col=col,
    )

fig_orateurs_time.update_layout(
    height=300 * rows,
    width=1200,
    title_text="Dynamique temporelle des mentions par orateur",
    showlegend=False,
    template="plotly_white",
)

for row in range(1, rows + 1):
    for col in range(1, cols + 1):
        fig_orateurs_time.update_yaxes(range=[0, max_y], row=row, col=col)

for col in range(1, cols + 1):
    fig_orateurs_time.update_xaxes(title_text="Date", row=rows, col=col)

for row in range(1, rows + 1):
    fig_orateurs_time.update_yaxes(title_text="Nombre de mentions", row=row, col=1)

fig_orateurs_time.show()


/Users/leo/anaconda3/envs/myenv_clone/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [15]:
fig_orateurs_time.write_html("../reports/figs/fig_orateurs_time.html")

In [16]:
# # BOF
# # Grouper par date et orateur
# # Top XX orateurs
# top_orateurs = df["Nom_orateur"].value_counts().index[:10].tolist()

# df_orateur_grouped = (
#     df[df["Nom_orateur"].isin(top_orateurs)]
#     .groupby([pd.Grouper(key="DateSeance_day", freq="W"), "Nom_orateur"])
#     .size()
#     .reset_index(name="mentions")
# )

# fig = px.line(
#     df_orateur_grouped,
#     x="DateSeance_day",
#     y="mentions",
#     color="Nom_orateur",
#     title="Évolution temporelle des mentions par orateur",
#     labels={"DateSeance_day": "Date", "mentions": "Nombre de mentions", "Nom_orateur": "Orateur"},
#     template="plotly_white",
# )

# fig.show()

### Genre

In [17]:
df["civ"] = df["civ"].replace({"M.": "Homme", "Mme": "Femme"})

In [21]:
df["civ"].value_counts()

civ
Homme    2348
Femme    1038
Name: count, dtype: int64

In [18]:
fig = px.bar(df["civ"].value_counts())
fig.update_layout(
    title="Répartition des genres (civ)", template="plotly_white", showlegend=False
)
fig.show()

In [19]:
# Grouper par semaine et genre
df_civ_grouped = (
    df.groupby([pd.Grouper(key="DateSeance_day", freq="W"), "civ"])
    .size()
    .reset_index(name="mentions")
)

fig_gender = px.line(
    df_civ_grouped,
    x="DateSeance_day",
    y="mentions",
    color="civ",
    title="Évolution temporelle des mentions selon le genre",
    labels={"DateSeance_day": "Date", "mentions": "Nombre de mentions", "civ": "Genre"},
    template="plotly_white",
)

fig_gender.show()

/Users/leo/anaconda3/envs/myenv_clone/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/Users/leo/anaconda3/envs/myenv_clone/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [20]:
fig_gender.write_html("../reports/figs/fig_gender.html")